# Tutorial SIMPY

En el tutorial de simpy que se encuentra en la página http://simpy.readthedocs.io/en/latest/simpy_intro/basic_concepts.html encontramos un ejemplo sencillo del tiempo que toma un automovil en estacionar y el tiempo que toma el mismo automovil en ser conducido.

In [1]:
import simpy
def car(env):
    while True:
        print('Inicia estacionamiento en %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)
        print('Inicia conduccion en %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)
env = simpy.Environment()
env.process(car(env))
env.run(until=15)

Inicia estacionamiento en 0
Inicia conduccion en 5
Inicia estacionamiento en 7
Inicia conduccion en 12
Inicia estacionamiento en 14


Mas adelante aparece una nueva versión del código con una serie de modificaciones, Car es una clase que se agrega que representa como tal el vehiculo que inicia la acción, tambien se ve la implementación de cola de ejecución de los eventos de los vehículos que se generan.

In [3]:
class Car(object):
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())        
    def run(self):
        while True:
            print('Inicia estacionamiento en %d' % self.env.now)
            charge_duration = 5
            yield self.env.process(self.charge(charge_duration))
            print('Comienza conduccion en %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)
    def charge(self, duration):
        yield self.env.timeout(duration)
env = simpy.Environment()
car = Car(env)
env.run(until=15)        

Inicia estacionamiento en 0
Comienza conduccion en 5
Inicia estacionamiento en 7
Comienza conduccion en 12
Inicia estacionamiento en 14


Para la tercera y última parte del tutorial vemos que se agrega una función llamada driver, tambien la implementación del tiempo de carga de la batería del vehículo y como afecta el funcionamiento de la cola cuando un vehiculo presenta este tipo de fallos.

In [4]:
def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()
class Car1(object):
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())
    def run(self):
        while True:
            print('Inicia estacionamiento en %d' % self.env.now)
            charge_duration = 5
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                print('Alto. Bateria incompleta')

            print('Inicia conduccion en %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)
    def charge(self, duration):
        yield self.env.timeout(duration)
env = simpy.Environment()
car = Car1(env)
env.process(driver(env, car))
env.run(until=15)

Inicia estacionamiento en 0
Alto. Bateria incompleta
Inicia conduccion en 3
Inicia estacionamiento en 5
Inicia conduccion en 10
Inicia estacionamiento en 12
